In [1]:
import pandas as pd

### Why do we care?

In [2]:
s = Series([1, 2, 3])
s

0    1
1    2
2    3
dtype: int64

In [3]:
s.reindex([1, 2, 3])

1    2.0
2    3.0
3    NaN
dtype: float64

In [4]:
s = Series([1, 2, 3], dtype='Int64')
s

0    1
1    2
2    3
dtype: Int64

In [5]:
rs = s.reindex([1, 2, 3])
rs

1      2
2      3
3    NaN
dtype: Int64

### Operations

In [54]:
rs.sum()

5

In [55]:
df = pd.DataFrame({'Int': s, 'int': [1, 2, 1]})
df

,Int,int
0,1,1
1,2,2
2,NaN,1


In [56]:
df.dtypes

Int    Int64
int    int64
dtype: object

In [57]:
df.groupby('int').Int.sum()

int
1    1
2    2
Name: Int, dtype: int64

In [58]:
df + 1

,Int,int
0,2,2
1,3,3
2,NaN,2


In [59]:
df.loc[2] = np.nan

In [60]:
df

,Int,int
0,1,1.0
1,2,2.0
2,NaN,NaN


In [61]:
df.fillna(0)

,Int,int
0,1,1.0
1,2,2.0
2,0,0.0


In [62]:
df.fillna(0).dtypes

Int      Int64
int    float64
dtype: object

### Implementation

In [63]:
s = df['Int']
s

0      1
1      2
2    NaN
Name: Int, dtype: Int64

In [64]:
s.values

IntegerArray([1, 2, nan], dtype='Int64')

In [65]:
s.values._data

array([1, 2, 1])

In [66]:
s.values._mask

array([False, False,  True])

In [67]:
s.nbytes

27

In [68]:
s.values._data.nbytes

24

In [69]:
s.values._mask.nbytes

3

### Interface

In [70]:
s.values.dropna()

IntegerArray([1, 2], dtype='Int64')

In [71]:
s.values.factorize()

(array([ 0,  1, -1]), IntegerArray([1, 2], dtype='Int64'))

In [72]:
s.values.isna()

array([False, False,  True])

In [73]:
s.values.take([0, 1, 2])

IntegerArray([1, 2, nan], dtype='Int64')

In [74]:
s.values.unique()

IntegerArray([1, 2, nan], dtype='Int64')